In [6]:
# Some manual transforms of key variables (e.g. demographics)
# Then focus on creating a party level dataset
    # Some attempt to clean up pf -> cses keys!
# Country level dataset
    # Some notes on those occasional double country ids





In [7]:
%%time
%matplotlib inline
%run Prosser_header.py {dataset_name} {df_list}


Toggle code

Wall time: 9 ms


In [8]:
dataset_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\datasets\\"
output_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\output\\"

In [9]:
CSES_int = pd.read_stata(dataset_folder + "CSES"+os.sep+"integrated 1-4"+os.sep+"cses_imd.dta")

def get_dta_meta(fn):
    reader = pd.io.stata.StataReader( fn )
    header = reader.variable_labels()
    labels = reader.value_labels()
    return header,labels

# pd.read_stata(dataset_folder + "CSES"+os.sep+"module 5"+os.sep+"cses5.dta")
header5,labels5 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 5"+os.sep+"cses5.dta")
header4,labels4 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 4"+os.sep+"cses4.dta")
header3,labels3 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 3"+os.sep+"cses3.dta")
header2,labels2 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 2"+os.sep+"cses2.dta")
header1,labels1 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 1"+os.sep+"cses1.dta")
headerINT,labelsINT = get_dta_meta(dataset_folder + "CSES"+os.sep+"integrated 1-4"+os.sep+"cses_imd.dta")

In [5]:
### WEIGHTS

#  | Sample weights include those intended to correct for unequal
#  | selection probabilities resulting from "booster" samples,
#  | procedures for selection within the household, non-response,
#  | as well as other features of the sample design.
#  |
#  | Demographic weights adjust sample distributions of socio-
#  | demographic characteristics to more closely resemble the
#  | characteristics of the population.
#  |
#  | Political weights reconcile discrepancies in the reported
#  | electoral behavior of the survey respondents from the
#  | official vote counts.

# they don't advise combining weights (I'm not 100% clear why)

# but you will need to standardise the weights
# - within election studies (already done for IMD weights)
# - between election studies to equalise their effective sample size

sample_size_per_ES = CSES_int["IMD1010_1"].groupby(CSES_int["IMD1003"]).sum()
sample_weights_ssadj = CSES_int["IMD1010_1"].values/sample_size_per_ES.loc[CSES_int["IMD1003"]].values

sample_size_per_ES = CSES_int["IMD1010_2"].groupby(CSES_int["IMD1003"]).sum()
demographic_weights_ssadj = CSES_int["IMD1010_2"].values/sample_size_per_ES.loc[CSES_int["IMD1003"]].values

sample_size_per_ES = CSES_int["IMD1010_3"].groupby(CSES_int["IMD1003"]).sum()
political_weights_ssadj = CSES_int["IMD1010_3"].values/sample_size_per_ES.loc[CSES_int["IMD1003"]].values

In [8]:
df = pd.DataFrame()

In [9]:
### HOUSEHOLD INCOME

imd_var = "IMD2006"

# CSES_int[imd_var].cat.categories
# check to see if ordinal/categorical


non_response_cats = ['6. [SEE ELECTION STUDY NOTES]', '7. VOLUNTEERED: REFUSED', "8. VOLUNTEERED: DON'T KNOW", '9. MISSING']

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
top_cat = CSES_int[imd_var].cat.remove_categories(non_response_cats).cat.categories[-1]
df[imd_var+"__"+top_cat] = CSES_int[imd_var].cat.remove_categories(non_response_cats).cat.codes.replace(-1,np.nan)

In [187]:
## AGE: Categorical

# 4000 non-responses

# categorical -> dummies

imd_var = "IMD2001_2"

non_response_cats = ['9997. REFUSED', "9998. DON'T KNOW",'9999. MISSING']
replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp


In [ ]:
## AGE: Numerical

imd_var = "IMD2001_1"
# 6000 non-responses

non_response_cats = ['9997. REFUSED', "9998. DON'T KNOW",'9999. MISSING']
replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
top_cat = CSES_int[imd_var].cat.remove_categories(non_response_cats).cat.categories[-1]
df[imd_var+"__"+str(top_cat)] = CSES_int[imd_var].replace(replace_dict).astype(float)

In [191]:
## Gender

imd_var = "IMD2002"
CSES_int[imd_var].cat.categories

non_response_cats = ['3. [SEE ELECTION STUDY NOTES]', '7. REFUSED', '9. MISSING']
replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

## could work this different for two-valued variables ...

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [194]:
## EDUCATION

# Reasonable question whether this is ordinal/categorical

imd_var = "IMD2003"
CSES_int[imd_var].cat.categories

non_response_cats = ['6. OTHER', '7. VOLUNTEERED: REFUSED', "8. VOLUNTEERED: DON'T KNOW", '9. MISSING']

replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [ ]:
    IMD2001_1    >>>         AGE OF RESPONDENT (IN YEARS)
    IMD2001_2    >>>         AGE OF RESPONDENT (IN CATEGORIES)
    IMD2002      >>>         GENDER
    IMD2003      >>>         EDUCATION
    IMD2004      >>>         MARITAL STATUS
    IMD2005      >>>         RELIGIOUS DENOMINATION 
    IMD2006      >>>         HOUSEHOLD INCOME
    IMD2007      >>>         RURAL OR URBAN RESIDENCE

In [196]:
## MARITAL STATUS

imd_var = "IMD2004"
CSES_int[imd_var].cat.categories

non_response_cats = ['5. [SEE ELECTION STUDY NOTES]','7. VOLUNTEERED: REFUSED', "8. VOLUNTEERED: DON'T KNOW", '9. MISSING']

replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [198]:
## RELIGIOUS DENOMINATION 

imd_var = "IMD2005"
CSES_int[imd_var].cat.categories

non_response_cats = ['97. VOLUNTEERED: REFUSED', "98. VOLUNTEERED: DON'T KNOW",'99. MISSING']

replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [202]:
## RURAL OR URBAN

imd_var = "IMD2007"
CSES_int[imd_var].cat.categories

non_response_cats = [ '7. VOLUNTEERED: REFUSED', "8. VOLUNTEERED: DON'T KNOW", '9. MISSING']

replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [30]:
cses_ids_to_pf = pd.read_csv(dataset_folder + "CSES"+os.sep+"Integration"+os.sep+"cses_ids_to_pf.csv")
cses_ids_to_pf = cses_ids_to_pf.set_index("Unnamed: 0")["0"]
cses_ids_to_pf = cses_ids_to_pf.dropna()

In [12]:
CSES_int["IMD3005_3"].value_counts()
# 9999999. MISSING                                           120168
# 9999997. VOLUNTEERED: REFUSED                                       1840
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)              1380
# 9999998. VOLUNTEERED: DON'T KNOW                                    1304
# 9999988. NONE OF THE CANDIDATES PARTIES                              517
# 9999990. OTHER LEFT WING CANDIDATE/PARTY                                   67
# 9999991. OTHER RIGHT WING CANDIDATE/PARTY                                  36
# 9999989. INDEPENDENT CANDIDATE                                             34



missing_dict = {9999999:np.nan,
                9999997:np.nan,
                9999992:np.nan,
                9999998:np.nan,
                9999988:np.nan,
                9999990:np.nan,
                9999991:np.nan,
                9999989:np.nan,
               }


party_ident = CSES_int["IMD3005_3"].apply(lambda x: int(x[0:7])).replace(missing_dict)


In [33]:
%%time
pf_party_ident = party_ident.apply(lambda x: cses_ids_to_pf.loc[x] if pd.notnull(x) and (x in cses_ids_to_pf.index) else np.nan)

Wall time: 8.4 s


In [35]:
pf_party_ident.value_counts()

1998.0    3547
432.0     3371
424.0     3293
1824.0    2775
1474.0    2773
          ... 
7028.0       1
7005.0       1
7004.0       1
7024.0       1
4410.0       1
Name: IMD3005_3, Length: 628, dtype: int64

In [39]:
mask = df[imd_var+"__"+top_cat].notnull() & pf_party_ident.notnull()
mask.sum()

128831

In [38]:
demographic_weights_ssadj.shape

(281083,)

In [44]:
party_id_cses = CSES_int["IMD3005_3"][mask].cat.remove_unused_categories()

In [46]:
df[imd_var+"__"+top_cat][mask].groupby(party_id_cses[mask]).mean().sort_values()

IMD3005_3
6080080. PHL - Abang Lingkod, Inc.                       0.0
6080038. PHL - Anakpawis                                 0.0
6040033. PER - Project Country                           0.0
7640008. THA - Motherland Party (Phak Matunhum Party)    0.5
4840016. MEX - Cardenista Party                          0.5
                                                        ... 
6080052. PHL - Cooperative NATCCO Party                  4.0
0080031. ALB - Agrarian Party                            4.0
0760012. BRA - Solidarity                                4.0
7100007. ZAF - Freedom Front Plus                        4.0
0560002. BEL - Socialist Party (Francophone)             4.0
Name: IMD2006__5. HIGHEST HOUSEHOLD INCOME QUINTILE, Length: 634, dtype: float64

In [48]:
sample_size_mask = df[imd_var+"__"+top_cat][mask].groupby(party_id_cses[mask]).count()>100

In [50]:
sample_size_mask.sum()

225

In [54]:
df[imd_var+"__"+top_cat][mask].groupby(party_id_cses[mask]).mean()[sample_size_mask].sort_values().tail(50)
# 0-4

IMD3005_3
2030005. CZE - Green Party                                                 2.233083
7050001. SVN - Slovenian Democratic Party                                  2.251656
2080006. DNK - Danish Social Liberal Party                                 2.254717
7920002. TUR - Republican People's Party                                   2.256338
0560004. BEL - Flemish Liberals and Democrats                              2.268116
2080007. DNK - Conservative People's Party                                 2.272727
3520001. ISL - Independence Party                                          2.285135
2760006. DEU - Free Democratic Party                                       2.295322
1580003. TWN - People First Party                                          2.296392
3760004. ISR - Energy                                                      2.309211
7520003. SWE - Liberal People's Party                                      2.321070
1240006. CAN - Progressive Conservative                           

In [53]:
# BUG!
# 0760002. BRA - Brazilian Social Dmeocratic Party                           2.420755

4.0

In [ ]:
IMD3002_PR_1 >>>         CURRENT PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 1
IMD3002_PR_2 >>>         CURRENT PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 2
IMD3002_LH_PL >>>        CURRENT LOWER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
IMD3002_LH_DC >>>        CURRENT LOWER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE
IMD3002_UH_PL >>>        CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
IMD3002_UH_DC_1 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 1
IMD3002_UH_DC_2 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 2
IMD3002_UH_DC_3 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 3
IMD3002_UH_DC_4 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 4
IMD3002_OUTGOV  >>>      CURRENT MAIN ELECTION: VOTE CHOICE - OUTGOING GOVERNMENT (INCUMBENT)
    
IMD3004_PR_1 >>>         PREVIOUS PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 1
IMD3004_PR_2 >>>         PREVIOUS PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 2
IMD3004_LH_PL >>>        PREVIOUS LOWER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
IMD3004_LH_DC >>>        PREVIOUS LOWER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE
IMD3004_UH_PL >>>        PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
IMD3004_UH_DC_1 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 1
IMD3004_UH_DC_2 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 2
IMD3004_UH_DC_3 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 3

In [64]:
cses_vote_vars = ["IMD3002_PR_1",
"IMD3002_PR_2",
"IMD3002_LH_PL",
"IMD3002_LH_DC",
"IMD3002_UH_PL",
"IMD3002_UH_DC_1",
"IMD3002_UH_DC_2",
"IMD3002_UH_DC_3",
"IMD3002_UH_DC_4",
"IMD3004_PR_1",
"IMD3004_PR_2",
"IMD3004_LH_PL",
"IMD3004_LH_DC",
"IMD3004_UH_PL",
"IMD3004_UH_DC_1",
"IMD3004_UH_DC_2",
"IMD3004_UH_DC_3",]

In [83]:
missing_values = ['9999988. NONE OF THE CANDIDATES/PARTIES',
       '9999989. INDEPENDENT CANDIDATE',
       '9999990. OTHER LEFT WING CANDIDATE/PARTY',
       '9999991. OTHER RIGHT WING CANDIDATE/PARTY',
       '9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)',
       '9999993. INVALID/BLANK BALLOT',
       '9999995. NOT APPLICABLE: NO DISTRICT CANDIDATE VOTE',
       '9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT',
       '9999995. NOT APPLICABLE: NOT A LIST SYSTEM',
       '9999995. NOT APPLICABLE: UNICAMERAL SYSTEM',
       '9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION',
       '9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION',
       '9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION / NO SECOND ROUND',
       '9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE',
       '9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR LIST VOTE',
       '9999997. VOLUNTEERED: REFUSED', "9999998. VOLUNTEERED: DON'T KNOW",
       '9999999. MISSING/ABSTAINED (DID NOT VOTE)']

In [66]:
stacked_vote_vars = CSES_int[cses_vote_vars].stack()

In [72]:
stacked_vote_vars.value_counts().sort_index().tail(20).index

Index(['8580005. URY - Popular Assembly', '8580006. URY - Civic Union',
       '9999988. NONE OF THE CANDIDATES/PARTIES',
       '9999989. INDEPENDENT CANDIDATE',
       '9999990. OTHER LEFT WING CANDIDATE/PARTY',
       '9999991. OTHER RIGHT WING CANDIDATE/PARTY',
       '9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)',
       '9999993. INVALID/BLANK BALLOT',
       '9999995. NOT APPLICABLE: NO DISTRICT CANDIDATE VOTE',
       '9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT',
       '9999995. NOT APPLICABLE: NOT A LIST SYSTEM',
       '9999995. NOT APPLICABLE: UNICAMERAL SYSTEM',
       '9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION',
       '9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION',
       '9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION / NO SECOND ROUND',
       '9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE',
       '9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR LIST VOTE',
       '9999997. VOLUNTEERED: REFUSED', '9999998. VOLUNTEER

In [76]:
CSES_int[cses_vote_vars].replace( dict(zip(missing_values,[np.nan]*len(missing_values)))).notnull().sum().sort_values()

IMD3004_UH_DC_3        58
IMD3002_UH_DC_4       176
IMD3002_UH_DC_3       609
IMD3004_UH_DC_2       822
IMD3004_UH_PL        1843
IMD3002_UH_DC_2      3873
IMD3004_UH_DC_1      5700
IMD3002_UH_PL        8989
IMD3004_PR_2        11766
IMD3002_PR_2        16430
IMD3002_UH_DC_1     25164
IMD3004_PR_1        38302
IMD3004_LH_DC       48732
IMD3002_PR_1        50059
IMD3002_LH_DC       80302
IMD3004_LH_PL       82896
IMD3002_LH_PL      132594
dtype: int64

In [82]:
# party_id_cses = CSES_int["IMD3005_3"][mask].cat.remove_unused_categories()

In [ ]:
## Same quick breakdown, but using a different vote variable
# IMD3002_LH_PL     highest num non-null values!

In [91]:
lower_house_party_list = CSES_int["IMD3002_LH_PL"][mask].replace(dict(zip(missing_values,[np.nan]*len(missing_values))))
lower_house_party_list.value_counts()

mask = df[imd_var+"__"+top_cat].notnull() & lower_house_party_list.notnull()

sample_size_mask = df[imd_var+"__"+top_cat][mask].groupby(lower_house_party_list[mask]).count()>100

In [93]:
df[imd_var+"__"+top_cat][mask].groupby(lower_house_party_list[mask]).mean()[sample_size_mask].sort_values().tail(10)
# 0-4

IMD3002_LH_PL
6160001. POL - Civic Platform                                   2.561028
7560021. CHE - Green Liberal Party                              2.638655
6040052. PER - National Unity                                   2.645161
5780002. NOR - Conservative Party                               2.654362
6160008. POL - Freedom Union                                    2.690476
2030010. CZE - TOP 09                                           2.719178
5540004. NZL - Act New Zealand                                  2.723247
2460003. FIN - National Coalition Party                         2.748503
1000001. BGR - Citizens for European Development of Bulgaria    2.796875
6040051. PER - Alliance for the Great Change                    2.818182
Name: IMD2006__5. HIGHEST HOUSEHOLD INCOME QUINTILE, dtype: float64

In [94]:
df[imd_var+"__"+top_cat][mask].groupby(lower_house_party_list[mask]).mean()[sample_size_mask].sort_values().head(10)
# 0-4

IMD3002_LH_PL
7640001. THA - For Thais Party                              1.058824
7100001. ZAF - African National Congress                    1.157343
2030004. CZE - Communist Party of Bohemia and Moravia       1.359909
6160024. POL - Self-Defense of the Republic Poland          1.567797
5540009. NZL - Alliance                                     1.622222
1520004. CHL - Socialist Party                              1.629310
5540003. NZL - New Zealand First                            1.629328
3920012. JPN - New Frontier Party                           1.636364
3000001. GRC - Pan-Hellenic Socialist Movement              1.643059
6430002. RUS - Communist Party of the Russian Federation    1.670554
Name: IMD2006__5. HIGHEST HOUSEHOLD INCOME QUINTILE, dtype: float64

In [ ]:
## do a version with weights added in, prob won't make much difference but just in case!

In [ ]:
## First try at building a Party Level dataset

In [95]:
cses_variable_types = pd.read_pickle(dataset_folder + "CSES"+os.sep+"Integration"+os.sep+"cses_variable_types.pkl")
cses_variable_types[cses_variable_types["variable type"]=="party"]

CSES_int["IMD5000_A"]

0                  0080002. ALB - Democratic Party
1                  0080002. ALB - Democratic Party
2                  0080002. ALB - Democratic Party
3                  0080002. ALB - Democratic Party
4                  0080002. ALB - Democratic Party
                            ...                   
281078    7100001. ZAF - African National Congress
281079    7100001. ZAF - African National Congress
281080    7100001. ZAF - African National Congress
281081    7100001. ZAF - African National Congress
281082    7100001. ZAF - African National Congress
Name: IMD5000_A, Length: 281083, dtype: category
Categories (107, object): [0080002. ALB - Democratic Party < 0320001. ARG - Front for the Victory < 0360001. AUS - Liberal Party < 0360003. AUS - Australian Labor Party ... 8260002. GBR - Labour < 8400001. USA - Republican Party < 8400002. USA - Democratic Party < 8580001. URY - Broad Front]

In [262]:
cses_variable_types[cses_variable_types["variable type"]=="party"].head(50)

,description,variable type
IMD5000_A,PARTY A IDENTIFIER - NUMERICAL,party
IMD5000_B,PARTY B IDENTIFIER - NUMERICAL,party
IMD5000_C,PARTY C IDENTIFIER - NUMERICAL,party
IMD5000_D,PARTY D IDENTIFIER - NUMERICAL,party
IMD5000_E,PARTY E IDENTIFIER - NUMERICAL,party
IMD5000_F,PARTY F IDENTIFIER - NUMERICAL,party
IMD5000_G,PARTY G IDENTIFIER - NUMERICAL,party
IMD5000_H,PARTY H IDENTIFIER - NUMERICAL,party
IMD5000_I,PARTY I IDENTIFIER - NUMERICAL,party
IMD5001_A,PERCENT VOTE - LOWER HOUSE - PARTY A,party


In [204]:
party_variables = cses_variable_types.index[cses_variable_types["variable type"]=="party"]

In [209]:
party_level = CSES_int[party_variables].groupby(CSES_int["IMD1003"]).first()

In [214]:
# party_level.stack().value_counts().head(50)

# 999. MISSING                                     2401
# 0.0                                              1456
# 996. NOT APPLICABLE: UNICAMERAL SYSTEM           1350
# 997. NOT APPLICABLE: NO UPPER HOUSE ELECTION     1074
# 99. MISSING                                       862
# 997. NOT APPLICABLE: NO PRESIDENTIAL ELECTION     689
# 996. NOT APPLICABLE: NO ROLE OF PRESIDENT         540
# 9999999. MISSING                                  357
# 997. NOT APPLICABLE: NO LOWER HOUSE ELECTION      288
# 97. NOT APPLICABLE                                 40

party_level

,IMD5000_A,IMD5000_B,IMD5000_C,IMD5000_D,IMD5000_E,IMD5000_F,IMD5000_G,IMD5000_H,IMD5000_I,IMD5001_A,...,IMD5029_I,IMD5031_A,IMD5031_B,IMD5031_C,IMD5031_D,IMD5031_E,IMD5031_F,IMD5031_G,IMD5031_H,IMD5031_I
IMD1003,,,,,,,,,,,,,,,,,,,,,
00802005. ALBANIA (2005),0080002. ALB - Democratic Party,0080001. ALB - Socialist Party,0080004. ALB - Republican Party,0080005. ALB - Social Democratic Party,0080003. ALB - Socialist Movement for Integration,0080030. ALB - New Democratic Party,0080031. ALB - Agrarian Party,0080011. ALB - Democratic Alliance Party,0080010. ALB - Christian Democratic Party,44.06,...,0,12,1,0,0,0,1,1,0,0
03202015. ARGENTINA (2015),0320001. ARG - Front for the Victory,0320002. ARG - Let's Change,0320003. ARG - United for a New Alternative,0320004. ARG - Workers' Left Front,0320005. ARG - Progressives,0320006. ARG - Federal Commitment,0320007. ARG - Republican Proposal,0320008. ARG - Radical Civic Union,0320009. ARG - Justicialist Party,37.4,...,0,0,15,0,0,0,0,11,4,0
03601996. AUSTRALIA (1996),0360003. AUS - Australian Labor Party,0360001. AUS - Liberal Party,0360002. AUS - National Party of Australia,0360004. AUS - Australian Democrats,0360005. AUS - Australian Greens,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,38.7,...,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING
03602004. AUSTRALIA (2004),0360001. AUS - Liberal Party,0360003. AUS - Australian Labor Party,0360005. AUS - Australian Greens,0360002. AUS - National Party of Australia,0360011. AUS - Family First Party,0360004. AUS - Australian Democrats,0360008. AUS - One Nation Party,9999999. MISSING,9999999. MISSING,40.47,...,999. MISSING,14,0,0,3,0,0,0,999. MISSING,999. MISSING
03602007. AUSTRALIA (2007),0360003. AUS - Australian Labor Party,0360001. AUS - Liberal Party,0360002. AUS - National Party of Australia,0360005. AUS - Australian Greens,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,43.38,...,999. MISSING,21,0,0,0,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84001996. UNITED STATES (1996),8400002. USA - Democratic Party,8400001. USA - Republican Party,8400004. USA - Reform Party,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,48.1,...,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING
84002004. UNITED STATES (2004),8400001. USA - Republican Party,8400002. USA - Democratic Party,8400004. USA - Reform Party,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,49.9,...,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING
84002008. UNITED STATES (2008),8400002. USA - Democratic Party,8400001. USA - Republican Party,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,9999999. MISSING,53.18,...,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING


In [230]:
# base variables for the full set of party variabes _A ... _I

party_base_variables = ['IMD5000', 'IMD5001', 'IMD5002', 'IMD5003', 'IMD5004', 'IMD5005', 'IMD5011', 'IMD5012', 'IMD5029', 'IMD5031']

In [231]:
# for base in party_base_variables:
#     for party in ["A","B","C","D","E","F","G","H","I"]:
#         party_level[base+"_"+party]

In [236]:
df = pd.melt( party_level[ search( party_level , 'IMD5000' ).index].reset_index(),
         id_vars = ["IMD1003"])
for base in party_base_variables[1:]:
    df[base] = pd.melt( party_level[ search( party_level , base ).index].reset_index(),
         id_vars = ["IMD1003"])["value"]
    

In [252]:
# 999. MISSING                                     2401
# 0.0                                              1456
# 996. NOT APPLICABLE: UNICAMERAL SYSTEM           1350
# 997. NOT APPLICABLE: NO UPPER HOUSE ELECTION     1074
# 99. MISSING                                       862
# 997. NOT APPLICABLE: NO PRESIDENTIAL ELECTION     689
# 996. NOT APPLICABLE: NO ROLE OF PRESIDENT         540
# 9999999. MISSING                                  357
# 997. NOT APPLICABLE: NO LOWER HOUSE ELECTION      288
# 97. NOT APPLICABLE                                 40

missing_vals = ["999. MISSING","996. NOT APPLICABLE: UNICAMERAL SYSTEM","997. NOT APPLICABLE: NO UPPER HOUSE ELECTION",
 "99. MISSING","997. NOT APPLICABLE: NO PRESIDENTIAL ELECTION","996. NOT APPLICABLE: NO ROLE OF PRESIDENT",
 "9999999. MISSING","997. NOT APPLICABLE: NO LOWER HOUSE ELECTION","97. NOT APPLICABLE",0]

replace_dict = dict(zip(missing_vals,[np.nan]*len(missing_vals)))

In [253]:
## bunch of 0s in IMD5029, IMD5031 (min. portfolios before/after by party) which should be missing (because party identifier columns registers missing!)
## -> minor bug

# 37602006. ISRAEL (2006)	IMD5000_G: IMD5031 -> 2 ??? BUG!

# 15201999. CHILE (1999)	IMD5000_E: IMD5005 -> .4
# 44001997. LITHUANIA (1997)	IMD5000_F	IMD5000_E: IMD5005 -> .9
# Maybe they included data before dropping parties by 

## safe to drop all rows with missing party identifier

discrepant_not_nulls = df[df["value"]=="9999999. MISSING"][party_base_variables[1:]].replace(replace_dict).notnull()
df[df["value"]=="9999999. MISSING"][ discrepant_not_nulls.any(axis=1) ]

,IMD1003,variable,value,IMD5001,IMD5002,IMD5003,IMD5004,IMD5005,IMD5011,IMD5012,IMD5029,IMD5031
720,15201999. CHILE (1999),IMD5000_E,9999999. MISSING,997. NOT APPLICABLE: NO LOWER HOUSE ELECTION,997. NOT APPLICABLE: NO LOWER HOUSE ELECTION,997. NOT APPLICABLE: NO UPPER HOUSE ELECTION,997. NOT APPLICABLE: NO UPPER HOUSE ELECTION,0.4,97. NOT APPLICABLE,97. NOT APPLICABLE,999. MISSING,999. MISSING
961,44001997. LITHUANIA (1997),IMD5000_F,9999999. MISSING,997. NOT APPLICABLE: NO LOWER HOUSE ELECTION,997. NOT APPLICABLE: NO LOWER HOUSE ELECTION,996. NOT APPLICABLE: UNICAMERAL SYSTEM,996. NOT APPLICABLE: UNICAMERAL SYSTEM,0.9,97. NOT APPLICABLE,97. NOT APPLICABLE,999. MISSING,999. MISSING
1019,72402000. SPAIN (2000),IMD5000_F,9999999. MISSING,999. MISSING,999. MISSING,999. MISSING,999. MISSING,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,13. REGIONAL PARTIES,99. MISSING,999. MISSING,999. MISSING
1119,37602006. ISRAEL (2006),IMD5000_G,9999999. MISSING,999. MISSING,999. MISSING,996. NOT APPLICABLE: UNICAMERAL SYSTEM,996. NOT APPLICABLE: UNICAMERAL SYSTEM,997. NOT APPLICABLE: NO PRESIDENTIAL ELECTION,99. MISSING,99. MISSING,0,2


In [259]:
df = df.drop(df[df["value"]=="9999999. MISSING"].index).drop("variable",axis=1).rename(columns={"value":"IMD5000"})
column_rename = {"IMD1003": "ELECTION STUDY ID",
 'IMD5000': 'PARTY IDENTIFIER',
 'IMD5001': 'PERCENT VOTE - LOWER HOUSE',
 'IMD5002': 'PERCENT SEATS - LOWER HOUSE',
 'IMD5003': 'PERCENT VOTE - UPPER HOUSE',
 'IMD5004': 'PERCENT SEATS - UPPER HOUSE',
 'IMD5005': 'PERCENT VOTE - PRESIDENT',
 'IMD5011': 'IDEOLOGICAL FAMILY',
 'IMD5012': 'LEFT-RIGHT',
 'IMD5029': 'NUMBER OF PORTFOLIOS BEFORE',
 'IMD5031': 'NUMBER OF PORTFOLIOS AFTER'}

df = df.rename(columns=column_rename)

In [290]:
df["CSES PARTY NUMBER"] = df["PARTY IDENTIFIER"].apply(lambda x: int(x[0:7]))
                                                       
df["COUNTRY"] = df["PARTY IDENTIFIER"].apply(lambda x: x[8:12])

df["PARTY NAME"] = df["PARTY IDENTIFIER"].apply(lambda x: x[15:])

df["ELECTION YEAR"] = df["ELECTION STUDY ID"].apply(lambda x: int(x[4:8]))
# pull out country code/year?
df

# basically done for party_level data

# ready to replace with party_ids

# would still need processing to numerical version

,ELECTION STUDY ID,PARTY IDENTIFIER,PERCENT VOTE - LOWER HOUSE,PERCENT SEATS - LOWER HOUSE,PERCENT VOTE - UPPER HOUSE,PERCENT SEATS - UPPER HOUSE,PERCENT VOTE - PRESIDENT,IDEOLOGICAL FAMILY,LEFT-RIGHT,NUMBER OF PORTFOLIOS BEFORE,NUMBER OF PORTFOLIOS AFTER,CSES PARTY NUMBER,COUNTRY,PARTY NAME,ELECTION YEAR
0,00802005. ALBANIA (2005),0080002. ALB - Democratic Party,44.06,40,996. NOT APPLICABLE: UNICAMERAL SYSTEM,996. NOT APPLICABLE: UNICAMERAL SYSTEM,997. NOT APPLICABLE: NO PRESIDENTIAL ELECTION,9. CONSERVATIVE PARTIES,6,0,12,80002,ALB,Democratic Party,2005
1,03202015. ARGENTINA (2015),0320001. ARG - Front for the Victory,37.4,43.85,32.72,54.17,997. NOT APPLICABLE: NO PRESIDENTIAL ELECTION,5. LEFT LIBERAL PARTIES,4,21,0,320001,ARG,Front for the Victory,2015
2,03601996. AUSTRALIA (1996),0360003. AUS - Australian Labor Party,38.7,33.1,36.1,38.1,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,4. SOCIAL DEMOCRATIC PARTIES,4.5,999. MISSING,999. MISSING,360003,AUS,Australian Labor Party,1996
3,03602004. AUSTRALIA (2004),0360001. AUS - Liberal Party,40.47,50,43.37,45,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,7. RIGHT LIBERAL PARTIES,6,14,14,360001,AUS,Liberal Party,2004
4,03602007. AUSTRALIA (2007),0360003. AUS - Australian Labor Party,43.38,55.33,40.3,45,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,4. SOCIAL DEMOCRATIC PARTIES,3,0,21,360003,AUS,Australian Labor Party,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,75202006. SWEDEN (2006),7520009. SWE - Feminist Party,999. MISSING,999. MISSING,996. NOT APPLICABLE: UNICAMERAL SYSTEM,996. NOT APPLICABLE: UNICAMERAL SYSTEM,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,99. MISSING,99. MISSING,999. MISSING,999. MISSING,7520009,SWE,Feminist Party,2006
1547,75202014. SWEDEN (2014),7520009. SWE - Feminist Party,3.12,0,996. NOT APPLICABLE: UNICAMERAL SYSTEM,996. NOT APPLICABLE: UNICAMERAL SYSTEM,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,90. OTHER,2,0,0,7520009,SWE,Feminist Party,2014
1554,76402011. THAILAND (2011),7640009. THA - Power of Choburi Party,0.55,1.4,997. NOT APPLICABLE: NO UPPER HOUSE ELECTION,997. NOT APPLICABLE: NO UPPER HOUSE ELECTION,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,98. NO IDEOLOGICAL FAMILY MENTIONED,5,0,1,7640009,THA,Power of Choburi Party,2011
1555,79202011. TURKEY (2011),7920012. TUR - Democratic Left Party,0.25,0,996. NOT APPLICABLE: UNICAMERAL SYSTEM,996. NOT APPLICABLE: UNICAMERAL SYSTEM,997. NOT APPLICABLE: NO PRESIDENTIAL ELECTION,4. SOCIAL DEMOCRATIC PARTIES,4,0,0,7920012,TUR,Democratic Left Party,2011


In [298]:
# cses_ids_to_pf

df["PARTYFACTS ID"] = df["CSES PARTY NUMBER"].apply(lambda x: cses_ids_to_pf.loc[x] if x in cses_ids_to_pf.index else np.nan)

# only 17 missing!

In [306]:
df[df["PARTYFACTS ID"].isnull()][["ELECTION YEAR","CSES PARTY NUMBER","PARTY IDENTIFIER"]]


,ELECTION YEAR,CSES PARTY NUMBER,PARTY IDENTIFIER
265,1997,4400102,4400102. LTU - V. Adamkus (Independent)
756,2004,3440026,3440026. HKG - April Fifth Action
869,2009,8580005,8580005. URY - Popular Assembly
896,2009,1520011,1520011. CHL - Peruvian Aprista Party
954,2004,4100020,4100020. KOR - National Integration 21
969,2012,4990037,4990037. MNE - Serbian Unity
978,2014,5540010,5540010. NZL - Internet MANA
1032,2011,7640006,7640006. THA - National Development of the Mot...
1033,2011,7920009,7920009. TUR - People's Voice Party
1143,2012,4990039,4990039. MNE - Force for Unity


In [335]:
df[df["CSES PARTY NUMBER"].apply(lambda x: x in [760008,760026])]

,ELECTION STUDY ID,PARTY IDENTIFIER,PERCENT VOTE - LOWER HOUSE,PERCENT SEATS - LOWER HOUSE,PERCENT VOTE - UPPER HOUSE,PERCENT SEATS - UPPER HOUSE,PERCENT VOTE - PRESIDENT,IDEOLOGICAL FAMILY,LEFT-RIGHT,NUMBER OF PORTFOLIOS BEFORE,NUMBER OF PORTFOLIOS AFTER,CSES PARTY NUMBER,COUNTRY,PARTY NAME,ELECTION YEAR,PARTYFACTS ID
1232,07602014. BRAZIL (2014),0760026. BRA - Progressive Republican Party,4.55,4.09,0.34,0,999. MISSING,9. CONSERVATIVE PARTIES,7,1,1,760026,BRA,Progressive Republican Party,2014,NaN


In [326]:
partyfacts_core_parties = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-core-parties.csv")
partyfacts_external_parties = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-external-parties.csv")
partyfacts_countries = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-countries.csv")
partyfacts_datasets = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-datasets.csv")

pf_country_to_code = partyfacts_countries[["country","name"]].set_index("name")["country"].apply(lambda x: x.lower())

county_iso = 'THA'

pfc = partyfacts_core_parties[partyfacts_core_parties["country"]==county_iso].sort_values(by='year_first')
pfc[-60:]

,country,partyfacts_id,technical,name_short,name,name_english,name_other,year_first,year_last,share,share_year,new,wikipedia,description,comment,created,modified
6308,THA,2950,7.0,etc,other,other,NaN,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-07-07 14:01:50.294939+00:00,2020-07-16 15:49:11.980429+00:00
6303,THA,4026,15.0,ally,alliance,alliance,NaN,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-09 13:49:47.326667+00:00,2020-07-16 15:49:12.339563+00:00
6307,THA,2951,8.0,indep,independent,independent,NaN,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-07-07 14:01:50.306055+00:00,2020-07-16 15:49:12.058479+00:00
6304,THA,3089,12.0,tech,technical,technical,NaN,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-09-05 07:46:45.481591+00:00,2020-07-16 15:49:12.307297+00:00
6306,THA,2953,10.0,unknown,unknown,unknown,NaN,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-07-07 14:01:50.333128+00:00,2020-07-16 15:49:12.089483+00:00
6305,THA,2954,11.0,1-perc,1 percent,1 percent,NaN,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-07-07 14:01:50.344653+00:00,2020-07-16 15:49:12.108457+00:00
6279,THA,5866,NaN,KR,Khana Ratsadon,People's Party,Thai: คณะราษฎร,1932,1946.0,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Khana_Ratsadon,leader support 1944 Pridi Banomyong (KR) – se...,NaN,2018-02-22 17:17:55.505051+00:00,2018-11-11 01:12:20.366984+00:00
6258,THA,6033,NaN,CF,Constitutional Front Party,Constitutional Front Party,NaN,1946,1947.0,NaN,NaN,NaN,NaN,leader support 1946 Thawan Thamrongnawasawat (...,NaN,2018-07-09 19:12:55.261285+00:00,2018-11-11 01:12:20.338850+00:00
6285,THA,7430,NaN,SP,Sahachip Party,Sahachip Party,NaN,1946,1946.0,76.00,1946.0,NaN,https://en.wikipedia.org/wiki/1946_Siamese_gen...,57 of 75 seats together with Constitutional Fr...,NaN,2019-09-04 14:43:57.675766+00:00,2019-09-04 14:43:57.675805+00:00
6259,THA,2327,NaN,PP,Phak Prachathipat,Democrat Party,Thai: พรรคประชาธิปัตย์,1946,NaN,51.90,1969.0,NaN,https://en.wikipedia.org/wiki/Democrat_Party_%...,NaN,NaN,2014-06-14 08:11:42.810147+00:00,2018-07-07 20:10:01.477217+00:00


In [313]:
# 	ELECTION YEAR	CSES PARTY NUMBER	PARTY IDENTIFIER
# 265	1997	4400102	4400102. LTU - V. Adamkus (Independent)
## this was a crazy election, no way to tie this guy to any party

# 756	2004	3440026	3440026. HKG - April Fifth Action
# https://en.wikipedia.org/wiki/April_Fifth_Action
# Quite small, did unusually well in 2004 (3.4%, 1 seat)

# 869	2009	8580005	8580005. URY - Popular Assembly
# https://en.wikipedia.org/wiki/Popular_Assembly
# 0.66 in 2009

# BUG!
# 896	2009	1520011	1520011. CHL - Peruvian Aprista Party
# https://en.wikipedia.org/wiki/American_Popular_Revolutionary_Alliance
# I can't see any such party in *CHILE*
# https://en.wikipedia.org/wiki/2009_Chilean_parliamentary_election
# But I do see a party with the same LH vote%/seat% and reasonable left-right position (5)
# https://en.wikipedia.org/wiki/Independent_Regionalist_Party
cses_ids_to_pf.loc[1520011] = 1748
    
# 954	2004	4100020	4100020. KOR - National Integration 21
# http://world.kbs.co.kr/service/news_view.htm?lang=e&Seq_Code=8381
# https://en.wikipedia.org/wiki/2004_South_Korean_legislative_election
# too small, .66% and only 1 seat

# 969	2012	4990037	4990037. MNE - Serbian Unity
# 3 party coalition, gets 1.45% of the vote and 0 seats

# 978	2014	5540010	5540010. NZL - Internet MANA
# https://en.wikipedia.org/wiki/Internet_Party_and_Mana_Movement

# 1032	2011	7640006	7640006. THA - National Development of the Mot...
# https://en.wikipedia.org/wiki/Chart_Pattana_Party_(2007)
cses_ids_to_pf.loc[7640006] = 2374


# 1033	2011	7920009	7920009. TUR - People's Voice Party
# https://en.wikipedia.org/wiki/People%27s_Voice_Party
# too small!

# 1143	2012	4990039	4990039. MNE - Force for Unity
# https://en.wikipedia.org/wiki/For_Unity

# 1317	2012	4990032	4990032. MNE - Albanian Coalition
# https://en.wikipedia.org/wiki/Albanian_Coalition
# both small temporary coalitions

# 1353	2014	6420300	6420300. ROU - Calin-Constantin-Anton Popescu-...
# https://en.wikipedia.org/wiki/Liberal_Reformist_Party_(Romania)
# Founded by the guy mentioned at the same time - reasonable to 
cses_ids_to_pf.loc[6420300] = 3223

# 1232	2014	760026	0760026. BRA - Progressive Republican Party
# https://en.wikipedia.org/wiki/Progressive_Republican_Party_(Brazil) (founded 1991)
# https://en.wikipedia.org/wiki/Republicans_(Brazil) (founded 2005)
# Name refers to PRP - but the figures refer to Republicans (Brazilian Republican Party (PRB))
cses_ids_to_pf.loc[760026] = 7079
## CSES has references as far back as 2002 .. and there's a separate listing for
# 0760008 Brazilian Republican Party

# Makes it look like the party level results are assigned to the wrong id
# Is it a mistake in the Party A identifier?
# https://en.wikipedia.org/wiki/2014_Brazilian_general_election
# They kind of look like they should be in the top 9...



# 1271	2002	2760024	2760024. DEU - Party Of The Rule Of Law Offensive
# 1272	2002	2760024	2760024. DEU - Party Of The Rule Of Law Offensive
# https://en.wikipedia.org/wiki/Party_for_a_Rule_of_Law_Offensive
# too short an existence/mostly restricted to Hamburg

# 1206	2011	7640007	7640007. THA - Peaceloving Party
# https://en.wikipedia.org/wiki/Rak_Santi_Party
# 1554	2011	7640009	7640009. THA - Power of Choburi Party
# https://en.wikipedia.org/wiki/Phalang_Chon_Party
# typo - shoudl be Cho*n*buri
# both small and confusing!

In [371]:
# do these evaluations differ between those "twice in same year" studies?

country_double_study = df["ELECTION STUDY ID"].apply(lambda x:x[3]!="0")
df[country_double_study].describe(include='all').loc["unique"]

# No - Belgium Flanders/Walloon really are different regions with different parties
# US(2012) is just weird
# Only the Germany "27622002. GERMANY (2002 Mail-Back)" are worth deleting

ELECTION STUDY ID                5
PARTY IDENTIFIER                25
PERCENT VOTE - LOWER HOUSE      21
PERCENT SEATS - LOWER HOUSE     20
PERCENT VOTE - UPPER HOUSE      16
PERCENT SEATS - UPPER HOUSE     12
PERCENT VOTE - PRESIDENT         7
IDEOLOGICAL FAMILY              12
LEFT-RIGHT                      10
NUMBER OF PORTFOLIOS BEFORE      4
NUMBER OF PORTFOLIOS AFTER       4
CSES PARTY NUMBER              NaN
COUNTRY                          3
PARTY NAME                      25
ELECTION YEAR                  NaN
PARTYFACTS ID                  NaN
Name: unique, dtype: object

In [374]:
df[country_double_study].sort_values(by="PARTY IDENTIFIER")

,ELECTION STUDY ID,PARTY IDENTIFIER,PERCENT VOTE - LOWER HOUSE,PERCENT SEATS - LOWER HOUSE,PERCENT VOTE - UPPER HOUSE,PERCENT SEATS - UPPER HOUSE,PERCENT VOTE - PRESIDENT,IDEOLOGICAL FAMILY,LEFT-RIGHT,NUMBER OF PORTFOLIOS BEFORE,NUMBER OF PORTFOLIOS AFTER,CSES PARTY NUMBER,COUNTRY,PARTY NAME,ELECTION YEAR,PARTYFACTS ID
10,05621999. BELGIUM-WALLONIA (1999),0560002. BEL - Socialist Party (Francophone),23.6,27.1,28.2,33.3,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,4. SOCIAL DEMOCRATIC PARTIES,99. MISSING,999. MISSING,999. MISSING,560002,BEL,Socialist Party (Francophone),1999,500.0
183,05611999. BELGIUM-FLANDERS (1999),0560003. BEL - Christian Democratic & Flemish,23.3,23.7,23.7,24,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,8. CHRISTIAN DEMOCRATIC PARTIES,5,999. MISSING,999. MISSING,560003,BEL,Christian Democratic & Flemish,1999,604.0
9,05611999. BELGIUM-FLANDERS (1999),0560004. BEL - Flemish Liberals and Democrats,23.4,25,24.2,24,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,7. RIGHT LIBERAL PARTIES,6,999. MISSING,999. MISSING,560004,BEL,Flemish Liberals and Democrats,1999,49.0
357,05611999. BELGIUM-FLANDERS (1999),0560005. BEL - Socialist Party Differently,16.2,16.2,14.6,16,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,4. SOCIAL DEMOCRATIC PARTIES,4,999. MISSING,999. MISSING,560005,BEL,Socialist Party Differently,1999,1680.0
531,05611999. BELGIUM-FLANDERS (1999),0560012. BEL - Flemish Block,15.9,16.2,14.8,16,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,13. REGIONAL PARTIES,9,999. MISSING,999. MISSING,560012,BEL,Flemish Block,1999,553.0
358,05621999. BELGIUM-WALLONIA (1999),0560013. BEL - Confederated Ecologists,17,15.7,19.7,20,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,1. ECOLOGY PARTIES,99. MISSING,999. MISSING,999. MISSING,560013,BEL,Confederated Ecologists,1999,1563.0
879,05611999. BELGIUM-FLANDERS (1999),0560015. BEL - People's Union,9.1,8.7,8.2,8,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,90. OTHER,4,999. MISSING,999. MISSING,560015,BEL,People's Union,1999,1424.0
184,05621999. BELGIUM-WALLONIA (1999),0560016. BEL - Liberal Reformist Party - Franc...,23.5,25.7,25.8,26.6,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,99. MISSING,99. MISSING,999. MISSING,999. MISSING,560016,BEL,Liberal Reformist Party - Francophone Democrat...,1999,554.0
705,05611999. BELGIUM-FLANDERS (1999),0560018. BEL - Live Differently - Flemish Ecol...,11.6,10,11.4,12,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,1. ECOLOGY PARTIES,2,999. MISSING,999. MISSING,560018,BEL,Live Differently - Flemish Ecologists,1999,528.0
706,05621999. BELGIUM-WALLONIA (1999),0560019. BEL - National Front,3.3,1.4,4,0,996. NOT APPLICABLE: NO ROLE OF PRESIDENT,10. NATIONAL PARTIES,99. MISSING,999. MISSING,999. MISSING,560019,BEL,National Front,1999,191.0


In [339]:
country_level_variables = cses_variable_types.index[cses_variable_types["variable type"]=="country"]

In [340]:
country_level_variables

Index(['IMD1003', 'IMD1004', 'IMD1006', 'IMD1006_UN', 'IMD1006_NAM',
       'IMD1008_YEAR', 'IMD1008_MOD_1', 'IMD1008_MOD_2', 'IMD1008_MOD_3',
       'IMD1008_MOD_4', 'IMD1009', 'IMD1011_M', 'IMD1011_D', 'IMD1011_Y',
       'IMD1012_M', 'IMD1012_D', 'IMD1012_Y', 'IMD5006_1', 'IMD5006_2',
       'IMD5007', 'IMD5008_1', 'IMD5008_2', 'IMD5009_1', 'IMD5009_2',
       'IMD5013', 'IMD5014', 'IMD5016_1', 'IMD5016_2', 'IMD5016_3',
       'IMD5016_4', 'IMD5017_1', 'IMD5017_2', 'IMD5017_3', 'IMD5017_4',
       'IMD5018_1', 'IMD5018_2', 'IMD5018_3', 'IMD5018_4', 'IMD5021_1',
       'IMD5021_2', 'IMD5021_3', 'IMD5021_4', 'IMD5022_1', 'IMD5022_2',
       'IMD5022_3', 'IMD5022_4', 'IMD5024_1', 'IMD5024_2', 'IMD5024_3',
       'IMD5025_1', 'IMD5025_2', 'IMD5025_3', 'IMD5026_1', 'IMD5026_2',
       'IMD5027', 'IMD5028', 'IMD5030', 'IMD5049', 'IMD5050_1', 'IMD5050_2',
       'IMD5050_3', 'IMD5051_1', 'IMD5051_2', 'IMD5051_3', 'IMD5052_1',
       'IMD5052_2', 'IMD5052_3', 'IMD5053_1', 'IMD5053_2', 'IMD5

In [344]:
country_level = CSES_int[country_level_variables].groupby("IMD1003").first()

# country_double_study = country_level.reset_index()["IMD1003"].apply(lambda x:x[3]!="0")

# we can drop those weird "two studies in one year" rows
country_level = country_level.drop(["05611999. BELGIUM-FLANDERS (1999)","27622002. GERMANY (2002 Mail-Back)"])

In [379]:
country_level[country_double_study.values].sort_index()

,IMD1004,IMD1006,IMD1006_UN,IMD1006_NAM,IMD1008_YEAR,IMD1008_MOD_1,IMD1008_MOD_2,IMD1008_MOD_3,IMD1008_MOD_4,IMD1009,...,IMD5056_1,IMD5056_2,IMD5056_3,IMD5057_1,IMD5057_2,IMD5057_3,IMD5058_1,IMD5058_2,IMD5059_1,IMD5059_2
IMD1003,,,,,,,,,,,,,,,,,,,,,
05611999. BELGIUM-FLANDERS (1999),BELF1999,0561,056,Belgium,1999,01. IN CSES MODULE 1,00. NOT IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,10. PARLIAMENTARY/LEGISLATIVE,...,0.57,1.76,0.9,10226419,10203008,10181245,10.28,999.00. MISSING,9.05,999.00. MISSING
05621999. BELGIUM-WALLONIA (1999),BELW1999,0562,056,Belgium,1999,01. IN CSES MODULE 1,00. NOT IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,10. PARLIAMENTARY/LEGISLATIVE,...,0.57,1.76,0.9,10226419,10203008,10181245,10.28,999.00. MISSING,9.05,999.00. MISSING
27612002. GERMANY (2002 Telephone),DEU12002,2761,276,Germany,2002,00. NOT IN CSES MODULE 1,01. IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,10. PARLIAMENTARY/LEGISLATIVE,...,1.35,1.28,-0.45,82488495,82349925,82211508,4.09,999.00. MISSING,3.38,999.00. MISSING
27622002. GERMANY (2002 Mail-Back),DEU22002,2762,276,Germany,2002,00. NOT IN CSES MODULE 1,01. IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,10. PARLIAMENTARY/LEGISLATIVE,...,1.35,1.28,-0.45,82488495,82349925,82211508,4.09,999.00. MISSING,3.38,999.00. MISSING
84022012. UNITED STATES (2012),USA_2012,8402,840,United States of America,2012,00. NOT IN CSES MODULE 1,00. NOT IN CSES MODULE 2,00. NOT IN CSES MODULE 3,01. IN CSES MODULE 4,12. PARLIAMENTARY/LEGISLATIVE AND PRESIDENTIAL,...,1.75,1.96,1.21,313873696,311582560,309326304,2.19,2.19,1.99,1.99


In [364]:
country_level

,IMD1004,IMD1006,IMD1006_UN,IMD1006_NAM,IMD1008_YEAR,IMD1008_MOD_1,IMD1008_MOD_2,IMD1008_MOD_3,IMD1008_MOD_4,IMD1009,...,IMD5056_1,IMD5056_2,IMD5056_3,IMD5057_1,IMD5057_2,IMD5057_3,IMD5058_1,IMD5058_2,IMD5059_1,IMD5059_2
IMD1003,,,,,,,,,,,,,,,,,,,,,
00802005. ALBANIA (2005),ALB_2005,0080,008,Albania,2005,00. NOT IN CSES MODULE 1,01. IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,10. PARLIAMENTARY/LEGISLATIVE,...,3.3,3.16,5.2,3011487,3026939,3039616,10.46,999.00. MISSING,3.75,999.00. MISSING
03202015. ARGENTINA (2015),ARG_2015,0320,032,Argentina,2015,00. NOT IN CSES MODULE 1,00. NOT IN CSES MODULE 2,00. NOT IN CSES MODULE 3,01. IN CSES MODULE 4,12. PARLIAMENTARY/LEGISLATIVE AND PRESIDENTIAL,...,23.75,40.72,23.88,43416756,42980024,42538304,3.4,3.4,3.52,3.48
03601996. AUSTRALIA (1996),AUS_1996,0360,036,Australia,1996,01. IN CSES MODULE 1,00. NOT IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,10. PARLIAMENTARY/LEGISLATIVE,...,2.7,2.27,1.02,18311000,18072000,17855000,3.21,999.00. MISSING,2.62,999.00. MISSING
03602004. AUSTRALIA (2004),AUS_2004,0360,036,Australia,2004,00. NOT IN CSES MODULE 1,01. IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,10. PARLIAMENTARY/LEGISLATIVE,...,3.31,3.11,2.86,20127400,19895400,19651400,3.17,999.00. MISSING,2.44,999.00. MISSING
03602007. AUSTRALIA (2007),AUS_2007,0360,036,Australia,2007,00. NOT IN CSES MODULE 1,00. NOT IN CSES MODULE 2,01. IN CSES MODULE 3,00. NOT IN CSES MODULE 4,10. PARLIAMENTARY/LEGISLATIVE,...,5.09,4.87,4.09,21072500,20697900,20394800,3.03,3.03,2.43,2.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84001996. UNITED STATES (1996),USA_1996,8400,840,United States of America,1996,01. IN CSES MODULE 1,00. NOT IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,12. PARLIAMENTARY/LEGISLATIVE AND PRESIDENTIAL,...,1.83,2.1,2.14,269394000,266278000,263126000,2.18,999.00. MISSING,2,999.00. MISSING
84002004. UNITED STATES (2004),USA_2004,8400,840,United States of America,2004,00. NOT IN CSES MODULE 1,01. IN CSES MODULE 2,00. NOT IN CSES MODULE 3,00. NOT IN CSES MODULE 4,12. PARLIAMENTARY/LEGISLATIVE AND PRESIDENTIAL,...,2.69,1.86,1.58,292805298,290107933,287625193,2.18,999.00. MISSING,2,999.00. MISSING
84002008. UNITED STATES (2008),USA_2008,8400,840,United States of America,2008,00. NOT IN CSES MODULE 1,00. NOT IN CSES MODULE 2,01. IN CSES MODULE 3,00. NOT IN CSES MODULE 4,12. PARLIAMENTARY/LEGISLATIVE AND PRESIDENTIAL,...,2.18,2.94,3.25,304375008,301580000,298592992,2.16,2.15,1.94,1.94


In [328]:
not_party_variables = cses_variable_types.index[cses_variable_types["variable type"]!="party"]

In [336]:
(CSES_int[not_party_variables]=="1520011. CHL - Peruvian Aprista Party").sum().sort_values()

IMD1001            0
IMD5006_2          0
IMD5007            0
IMD5008_1          0
IMD5008_2          0
                  ..
IMD3004_UH_PL      0
IMD3004_UH_DC_1    0
IMD3003_PR_1       0
IMD5059_2          0
IMD3002_LH_PL      5
Length: 186, dtype: int64

In [337]:
CSES_int[CSES_int["IMD3002_LH_PL"]=="1520011. CHL - Peruvian Aprista Party"]["IMD1008_YEAR"]

62670    2009
62679    2009
62681    2009
62777    2009
63131    2009
Name: IMD1008_YEAR, dtype: int16

In [260]:
# df[df["IMD1003"]=="84022012. UNITED STATES (2012)"]

#          | [ELECTION STUDY NOTES] - UNTIED STATES (2012): IMD5005_C -
#          |                                                IMD5005_E
#          |
#          | Data for Parties G (Libertarian Party, LP), H (Green Party, 
#          | GPUS) and I (Constitution Party, Con) are listed in slots C
#          | (Libertarian Party, LP), D (Green Party, GPUS) and E 
#          | (Constitution Party, Con). 

## Looks fine to me - reference to original module data?

In [99]:
CSES_int["IMD1008_YEAR"]

0         2005
1         2005
2         2005
3         2005
4         2005
          ... 
281078    2014
281079    2014
281080    2014
281081    2014
281082    2014
Name: IMD1008_YEAR, Length: 281083, dtype: int16

In [ ]:
## Year x Party


# party_A_identifier, year, party_A_left-right-pos



In [104]:
CSES_int[["IMD5000_A","IMD1008_YEAR","IMD5012_A",]]

,IMD5000_A,IMD1008_YEAR,IMD5012_A
0,0080002. ALB - Democratic Party,2005,6
1,0080002. ALB - Democratic Party,2005,6
2,0080002. ALB - Democratic Party,2005,6
3,0080002. ALB - Democratic Party,2005,6
4,0080002. ALB - Democratic Party,2005,6
...,...,...,...
281078,7100001. ZAF - African National Congress,2014,3
281079,7100001. ZAF - African National Congress,2014,3
281080,7100001. ZAF - African National Congress,2014,3
281081,7100001. ZAF - African National Congress,2014,3


In [110]:
["99. MISSING","97. NOT APPLICABLE"]

CSES_int["IMD5012_A"].value_counts().sort_index()

00. LEFT               2000
1.0                    4738
2.0                    4141
3.0                   44590
4.0                   44562
4.5                    2971
5.0                   36751
6.0                   37612
7.0                   62527
8.0                   16402
9.0                   11332
97. NOT APPLICABLE     1009
99. MISSING           12448
Name: IMD5012_A, dtype: int64

In [111]:
CSES_int["IMD5012_B"].value_counts().sort_index()

# dubious - check originals!

# Some collaborators have reported sub-incremental values (i.e.
#          | 4.5, 7.5, etc.).

00. LEFT               1496
1.0                    1000
2.0                    4641
3.0                   35066
4.0                   53598
5.0                   37917
5.5                    1798
6.0                   25859
7.0                   56581
8.0                   44295
9.0                    5375
97. NOT APPLICABLE     1009
99. MISSING           12448
Name: IMD5012_B, dtype: int64

In [154]:
### Left-Right Party_A

for party in ["A","B","C","D","E","F","G","H","I"]:

    imd_var = "IMD5012_"+party

    # CSES_int[imd_var].cat.categories
    # check to see if ordinal/categorical


    non_response_cats = ["99. MISSING","97. NOT APPLICABLE"]
    replace_dict = {"99. MISSING":np.nan,"97. NOT APPLICABLE":np.nan,'00. LEFT':0,'10. RIGHT':10}


    df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
    top_cat = '10. RIGHT'
    df[imd_var+"__"+top_cat] = CSES_int[imd_var].replace(replace_dict).astype(float)

In [161]:
for party in ["A","B","C","D","E","F","G","H","I"]:

    imd_var = "IMD5012_"+party

    mask = df[imd_var+"__"+top_cat].notnull() & (CSES_int["IMD5000_"+party]!="9999999. MISSING")

    sample_size_mask = df[imd_var+"__"+top_cat][mask].groupby(CSES_int["IMD5000_"+party][mask]).count()>100

    display( df[imd_var+"__"+top_cat][mask].groupby(CSES_int["IMD5000_"+party][mask]).mean()[sample_size_mask].sort_values() )

IMD5000_A
1120001. BLR - Communist Party of Belarus                   0.000000
8040010. UKR - Communist Party of Ukraine                   1.000000
6430002. RUS - Communist Party of the Russian Federation    1.000000
3000003. GRC - Coalition of the Radical Left                2.000000
7050016. SVN - Zoran Jankovic List - Positive Slovenia      2.000000
                                                              ...   
7560001. CHE - Swiss People's Party                         8.398421
2030002. CZE - Civic Democratic Party                       8.619622
4990011. MNE - Coalition 'For a European Montenegro'        9.000000
6040001. PER - Popular Force                                9.000000
1520002. CHL - Independent Democratic Union                 9.000000
Name: IMD5012_A__10. RIGHT, Length: 101, dtype: float64

IMD5000_B
6430002. RUS - Communist Party of the Russian Federation    0.0
3800011. ITA - Democrats of the Left                        2.0
3000003. GRC - Coalition of the Radical Left                2.0
4840003. MEX - Democratic Revolution Party                  2.0
1120004. BLR - Agrarian Party                               2.0
                                                           ... 
1520002. CHL - Independent Democratic Union                 8.0
5780002. NOR - Conservative Party                           8.0
2030002. CZE - Civic Democratic Party                       9.0
6040001. PER - Popular Force                                9.0
2500007. FRA - National Front                               9.0
Name: IMD5012_B__10. RIGHT, Length: 109, dtype: float64

IMD5000_C
2030004. CZE - Communist Party of Bohemia and Moravia     0.716564
3000004. GRC - Communist Party of Greece                  1.000000
3920004. JPN - Japanese Communist Party                   1.000000
4990003. MNE - Socialist People's Party                   1.000000
6160017. POL - Palikots Movement                          1.000000
                                                           ...    
2460003. FIN - National Coalition Party                   9.000000
2500007. FRA - National Front                            10.000000
3000006. GRC - Golden Dawn                               10.000000
1120017. BLR - Christian Conservative Party              10.000000
2030010. CZE - TOP 09                                    10.000000
Name: IMD5012_C__10. RIGHT, Length: 121, dtype: float64

IMD5000_D
3920004. JPN - Japanese Communist Party             0.567458
0320004. ARG - Workers' Left Front                  1.000000
0360005. AUS - Australian Greens                    1.000000
2500006. FRA - Left Front                           1.000000
3520005. ISL - Left-Green Movement                  1.642011
                                                     ...    
2030010. CZE - TOP 09                               9.000000
5540004. NZL - Act New Zealand                      9.000000
3480004. HUN - Hungarian Justice and Life Party     9.000000
1120027. BLR - The BNF Party                        9.000000
2500007. FRA - National Front                      10.000000
Name: IMD5012_D__10. RIGHT, Length: 114, dtype: float64

IMD5000_E
8580005. URY - Popular Assembly                  0.0
2500009. FRA - Workers' Struggle                 0.0
3440026. HKG - April Fifth Action                1.0
4100015. KOR - Democratic Labor Party            1.0
2500004. FRA - French Communist Party            1.0
                                                ... 
3920007. JPN - Japan Restoration Party           9.0
7030012. SVK - People's Party - Our Slovakia    10.0
0560012. BEL - Flemish Block                    10.0
3000006. GRC - Golden Dawn                      10.0
7050008. SVN - Slovenian National Party         10.0
Name: IMD5012_E__10. RIGHT, Length: 115, dtype: float64

IMD5000_F
1520006. CHL - Communist Party of Chile                      0.0
6200007. PRT - Portugese Communist Worker's Party            0.0
3440006. HKG - Labour Party                                  1.0
3800013. ITA - Communist Refoundation Party                  1.0
6160007. POL - Labour Union                                  1.0
                                                            ... 
4280014. LVA - Latvian Association of the Regions            9.0
1120005. BLR - United Civil Party                            9.0
6160014. POL - League of Polish Families                     9.0
3760050. ISR - National Union - National Religious Party     9.0
7050008. SVN - Slovenian National Party                     10.0
Name: IMD5012_F__10. RIGHT, Length: 108, dtype: float64

IMD5000_G
2500014. FRA - Revolutionary Communist League           0.0
4840011. MEX - Possible Mexico                          1.0
3000004. GRC - Communist Party of Greece                1.0
5540011. NZL - MANA Movement                            1.0
6040007. PER - Direct Democracy                         1.0
                                                       ... 
2760009. DEU - National Democratic Party Of Germany     9.0
6160004. POL - Solidarity Electoral Action              9.0
7920015. TUR - Great Union Party                        9.0
6160018. POL - Congress of the New Right               10.0
1000007. BGR - Attack                                  10.0
Name: IMD5012_G__10. RIGHT, Length: 72, dtype: float64

IMD5000_H
0400006. AUT - Communist Party of Austria                                 1.000000
5780010. NOR - Red Electoral Alliance                                     1.000000
5780009. NOR - Red Party                                                  1.000000
2080005. DNK - United List - Red-Greens                                   1.000000
4840008. MEX - Social Democratic Party                                    1.000000
8400005. USA - Green Party                                                2.000000
6430005. RUS - Pensioners Party                                           2.000000
5540009. NZL - Alliance                                                   2.000000
3920006. JPN - Social Democratic Party                                    2.000000
3760004. ISR - Energy                                                     2.000000
6160068. POL - Social Democracy of Poland                                 3.000000
2760014. DEU - Pirates Party                                              3.0

IMD5000_I
4840004. MEX - Labor Party                            1.000000
5780009. NOR - Red Party                              1.000000
3760010. ISR - United Arab List                       1.000000
2500004. FRA - French Communist Party                 1.000000
6160007. POL - Labour Union                           1.000000
6200026. PRT - Humanist Party                         2.000000
7520009. SWE - Feminist Party                         2.000000
0560013. BEL - Confederated Ecologists                3.000000
3920005. JPN - Green Wind                             3.000000
6080009. PHL - People's Reform Party                  4.000000
4990007. MNE - Croatian Civic Initiative              4.000000
1910011. HRV - Istrian Democratic Assembly            4.000000
1520007. CHL - Radical Social-democratic Party        4.000000
1000009. BGR - Movement 21                            4.000000
7240012. ESP - Union, Progress and Democracy          4.000000
4840026. MEX - Mexican Liberal Party         

In [158]:
df[imd_var+"__"+top_cat][mask].groupby(CSES_int["IMD5000_A"][mask]).mean()[sample_size_mask].sort_values()

IMD5000_A
1120001. BLR - Communist Party of Belarus                   0.000000
8040010. UKR - Communist Party of Ukraine                   1.000000
6430002. RUS - Communist Party of the Russian Federation    1.000000
3000003. GRC - Coalition of the Radical Left                2.000000
7050016. SVN - Zoran Jankovic List - Positive Slovenia      2.000000
                                                              ...   
7560001. CHE - Swiss People's Party                         8.398421
2030002. CZE - Civic Democratic Party                       8.619622
4990011. MNE - Coalition 'For a European Montenegro'        9.000000
6040001. PER - Popular Force                                9.000000
1520002. CHL - Independent Democratic Union                 9.000000
Name: IMD5012_A__10. RIGHT, Length: 101, dtype: float64

9999999. MISSING                                        210169
5280007. NLD - Reformed Political Party                   6086
0760009. BRA - Democrats                                  3136
5540006. NZL - United Future New Zealand                  2598
0760046. BRA - Liberal Party                              2514
7520009. SWE - Feminist Party                             2379
6420020. ROU - Democratic Liberal Party                   2283
0560013. BEL - Confederated Ecologists                    2225
2080011. DNK - Centre Democrats                           2026
2500018. FRA - Radical Party of the Left                  2000
0760010. BRA - Brazilian Labor Party                      2000
4840026. MEX - Mexican Liberal Party                      1991
3920005. JPN - Green Wind                                 1937
8400006. USA - Constitution Party                         1929
2760009. DEU - National Democratic Party Of Germany       1889
6160007. POL - Labour Union                            